In [2]:
print("OK")

OK


In [3]:
%pwd

'e:\\code\\Medical-Chatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'e:\\code\\Medical-Chatbot'

In [6]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("data/")

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#pip install --upgrade huggingface_hub

In [13]:
#pip install --upgrade jupyter ipywidgets

In [14]:
#pip install --upgrade huggingface_hub transformers tokenizers sentence-transformers

In [15]:
import sentence_transformers

In [16]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
#pip install langchain-huggingface

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

In [20]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [21]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
#HUGGING_FACE_API_KEY=os.environ.get('HUGGING_FACE_API_KEY')

In [32]:
#pip uninstall pinecone-plugin-inference

In [33]:
pip install pinecone 

Note: you may need to restart the kernel to use updated packages.


In [34]:
#pip install --upgrade pinecone

In [35]:
import pinecone

In [37]:
#Initializing the Pinecone
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name="medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'X-Cloud-Trace-Context': 'cc6885bd5fa569bdc9209c8e5eb1fed8', 'Date': 'Tue, 04 Mar 2025 15:51:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [38]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY
#os.environ["HUGGING_FACE_API_KEY"]=HUGGING_FACE_API_KEY

In [ ]:
#Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [49]:
#Load Existing Index
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [50]:
docsearch

In [51]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [52]:
retrieved_docs=retriever.invoke("What are Allergies?")

In [53]:
retrieved_docs

[Document(id='6004e609-68a0-44c6-88ef-f8ccc86cb5a4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 129.0, 'page_label': '130', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depend on the tissues most sensi-\ntive to the allergen and whether the allergen spread sys-'),
 Document(id='b300a4c2-2c9d-4d1e-b8ad-1faeb7cb3d9b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 135.0, 'page_label': '136', 'producer':

In [ ]:
#from langchain_openai import OpenAI
#llm= OpenAI(temperature=0.4,max_tokens=500)

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved conetxt to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [40]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

NameError: name 'llm' is not defined

In [ ]:
response=rag_chain.invoke({"input": "What are Allergies?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [41]:
from langchain import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
# HuggingFace API Key
huggingface_key = ""

In [43]:
from huggingface_hub import login
login(huggingface_key)

In [ ]:
pip install langchain langchain-community langchain-huggingface faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [44]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

In [54]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

# LLM Model
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/google/flan-t5-large",
    huggingfacehub_api_token=huggingface_key,
    temperature=0.5,
    max_new_tokens=256
)

# System Prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n{context}"
)

# Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Chain Pipeline
question_answer_chain = RunnablePassthrough() | prompt | llm

# Retrieval Chain
retriever_chain = retriever | question_answer_chain

# Query 🔥
response = retriever_chain.invoke({"input": "What are Acne?"})
print(response)


AttributeError: 'dict' object has no attribute 'replace'

In [55]:
# LLM Model
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    huggingfacehub_api_token=huggingface_key
)

system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved conetxt to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)



In [56]:
response=rag_chain.invoke({"input": "What are ACNE?Give medicines also."})
print(response["answer"])

c:\Users\sombi\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Acne vulgaris affecting a woman’s face.
